In [175]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.layers import Dropout


In [220]:
# Read the Excel file into a pandas DataFrame
df = pd.read_csv('Dataset/Text_Emotion_Data.csv')

# Define a function to tokenize text into word sequences and remove stopwords
def tokenize_text(text):
    # Remove non-letter characters using the defined pattern
    pattern = r'[^a-zA-Z]'
    text = re.sub(pattern, ' ', text)
    # Convert the text to lowercase
    text = text.lower()
    # Tokenize the text into word sequences
    word_sequences = text.split()
    # Remove stopwords using the provided list
    with open('Dataset/stopwords.txt', 'r') as f:
        stopwords = f.read().splitlines()
    # Remove words with length less than or equal to 2
    word_sequences = [word for word in word_sequences if (word not in stopwords and len(word) > 2)]
    return word_sequences

# Tokenize each row of the text column into word sequences using the defined function
df['word_sequences'] = df['Text'].apply(tokenize_text)

# Find the maximum length of a word sequence
max_len = max(df['word_sequences'].apply(len))

# Define a function to pad the sequences to the maximum length
def pad_sequence(sequence):
    padded_sequence = sequence[:max_len] + ['']*(max_len-len(sequence))
    return padded_sequence

# Pad each sequence to the maximum length
df['word_sequences'] = df['word_sequences'].apply(pad_sequence)

# Combine all word sequences into a single list
all_sequences = []
for seq in df['word_sequences']:
    all_sequences.append(seq)

# Create a dictionary with unique words as keys and their corresponding index as values
word_dict = {}
index = 0
for seq in all_sequences:
    for word in seq:
        if word not in word_dict:
            word_dict[word] = index
            index += 1

# Convert each word sequence into a numerical vector with the corresponding index in the dictionary
num_vectors = []
for sequence in all_sequences:
    vector = []
    for word in sequence:
        if word in word_dict:
            index = word_dict[word]
            vector.append(index)
    num_vectors.append(vector)

# Convert the list of numerical vectors into a numpy array
X = np.array(num_vectors)
# binrise the data 
print(X.shape)
NX = []
for i in  range (len(X)):
    row = []
    for j in range (len(X[i])):
        idx = np.binary_repr(X[i][j], width=14)
        row.append(idx)
    NX.append(row)
    
NX = np.array(NX)
X = NX


for i in range (len(X)):
    for j in range(len(X[i])):
        X[i][j] = np.int32(X[i][j])
        
        
X = X.astype(np.int)
print(type(X[20][6]))
# Define the labels
labels = df['Label'].values
classes = np.unique(labels)
label_map = {label: i for i, label in enumerate(classes)}
y = np.array([label_map[label] for label in labels])

# Split last 150 text of each class for the test dataset
test_data = []
for c in classes:
    class_data = [(X[i], y[i]) for i in range(len(X)) if y[i] == label_map[c]]
    test_data.extend(class_data[-150:])

# Use the rest of the data for training
train_data = []
for i in range(len(X)):
    found = False
    for j in range(len(test_data)):
        if all(X[i] == test_data[j][0]) and y[i] == test_data[j][1]:
            found = True
            break
    if not found:
        train_data.append((X[i], y[i]))

# Separate the input features and labels for the training and test sets
X_train, y_train = zip(*train_data)
X_test, y_test = zip(*test_data)


X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)



(3250, 27)


/var/folders/gv/29bx0dbj5fvf2ql1sq4xwxsh0000gn/T/ipykernel_92883/2724749026.py:79: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = X.astype(np.int)


<class 'numpy.int64'>


In [221]:
def is_array_numeric(data):
    data_type = data.dtype
    is_numeric = np.issubdtype(data_type, np.number) or np.issubdtype(data_type, np.floating)
    return is_numeric

# Example usage
print(is_array_numeric(X_train))

True


In [222]:
type(X_train[20][6])

numpy.int64

In [223]:
X_train[20][6]

10011

In [224]:
print('X_train shape :', X_train.shape)
print('X_test shape :' ,X_test.shape)
print('y_train shape :' ,y_train.shape)
print('y_test shape :', y_test.shape)
print('word_dict len :', len(word_dict))
print('max_len  :' ,max_len) 
y_train[20]

X_train shape : (2500, 27)
X_test shape : (750, 27)
y_train shape : (2500,)
y_test shape : (750,)
word_dict len : 6623
max_len  : 27


1

In [225]:
# Reshape the input data to have a third dimension
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [226]:
print('X_train shape :', X_train.shape)
print('X_test shape :' ,X_test.shape)
print('y_train shape :' ,y_train.shape)
print('y_test shape :', y_test.shape)
print('word_dict len :', len(word_dict))
print('max_len  :' ,max_len) 
# X_train[20]

X_train shape : (2500, 27, 1)
X_test shape : (750, 27, 1)
y_train shape : (2500,)
y_test shape : (750,)
word_dict len : 6623
max_len  : 27


In [227]:
y_train[20]

1

In [228]:
# Define the model architecture
model = Sequential()
model.add(SimpleRNN(64, input_shape=(max_len, 1), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(classes), activation='relu'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Convert y_train to one-hot encoding
# y_train = to_categorical(y_train, num_classes=5)
# y_test= to_categorical(y_test, num_classes=5)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=23)

# Evaluate the model on the train dataset
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print('Train Loss:', train_loss)
print('Train Accuracy:', train_acc)

# Evaluate the model on the test dataset
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)


Epoch 1/20
109/109 [==============================] - 1s 4ms/step - loss: 8.1342 - accuracy: 0.1988
Epoch 2/20
109/109 [==============================] - 0s 4ms/step - loss: 2.9418 - accuracy: 0.2008
Epoch 3/20
109/109 [==============================] - 0s 4ms/step - loss: 1.9333 - accuracy: 0.1972
Epoch 4/20
109/109 [==============================] - 0s 4ms/step - loss: 1.7617 - accuracy: 0.2004
Epoch 5/20
109/109 [==============================] - 0s 4ms/step - loss: 1.7085 - accuracy: 0.1996
Epoch 6/20
109/109 [==============================] - 0s 4ms/step - loss: 1.6497 - accuracy: 0.2012
Epoch 7/20
109/109 [==============================] - 0s 4ms/step - loss: 1.6772 - accuracy: 0.1992
Epoch 8/20
109/109 [==============================] - 0s 4ms/step - loss: 1.6355 - accuracy: 0.1996
Epoch 9/20
109/109 [==============================] - 0s 4ms/step - loss: 1.6418 - accuracy: 0.2000
Epoch 10/20
109/109 [==============================] - 0s 4ms/step - loss: 1.6284 - accuracy: 0.2000